In [53]:
import requests
import pandas as pd
import json
import psycopg2
import time

In [44]:
# connect postgresql
def get_postgresql_connect():
    conn = psycopg2.connect(host="127.0.0.1",user="postgres",password="123456",database="aim_new_final")
    return conn

def query_houses():
    
    conn= get_postgresql_connect()
    cur = conn.cursor()
    sql = " select * from t_houses "
    cur.execute(sql)
    
    rows = cur.fetchall()
        
    conn.close()
    
    return rows

def insert_houses(house_list):
    conn= get_postgresql_connect()
    cur = conn.cursor()
    
    sql = """
        INSERT INTO public.t_houses(property_id, prop_type, beds, baths_full, full_json, county, state)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
    
    for h in house_list:
        
        asjson = json.dumps(h)
        
        if 'property_id' in h and 'prop_type' in h:
            
            beds = 0
            if 'beds' in h:
                beds = h['beds']
                
            baths_full = 0
            if 'baths_full' in h:
                baths_full = h['baths_full']
            
            county = ''
            state = ''
            
            if 'address' in h:
                address = h['address']
                
                if 'county' in address:
                    county = address['county']
                if 'state' in address:
                    state = address['state']
                
            values = (h['property_id'], h['prop_type'], beds, baths_full, asjson, county, state)
        

        cur.execute(sql, values)
        
    conn.commit()
    print("Records created successfully")
    conn.close()

    
def query_history_prices():
    
    conn= get_postgresql_connect()
    cur = conn.cursor()
    sql = " select * from t_house_history_prices "
    cur.execute(sql)
    
    rows = cur.fetchall()
    
    conn.close()
    
    return rows
    
def insert_history_prices(property_id, history_list):
    conn= get_postgresql_connect()
    cur = conn.cursor()
    
    sql = """
        INSERT INTO t_house_history_prices(property_id, price, price_sqft, price_event, price_event_date, sqft)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
    # {'event_name': 'Sold', 'date': '2020-12-10T17:00:00Z', 'price': 490000, 'price_changed': 0, 
    # 'sqft': 1148, 'datasource_name': 'StatenIsland', 'source': 'MLS #1138690'}
    for h in history_list:
        
        price_sqft = 0
        
        if h['sqft'] and h['price'] and h['sqft'] > 0:
            price_sqft = h['price'] / h['sqft']
        
        values = (property_id, h['price'], price_sqft, h['event_name'], h['date'], h['sqft'])

        cur.execute(sql, values)
        
    conn.commit()
    print("Records created successfully")
    conn.close()
#query_houses()

In [10]:
# convert a json string into a Python object using the json.loads() function
def save_to_db(json_str):
    
    all_json = json.loads(json_str)
    house_list = all_json['properties']
    
    insert_houses(house_list)

# save_to_db(json_test)

In [11]:
def find_list():
    url = "https://realtor.p.rapidapi.com/properties/v2/list-sold"
    
    targets = [#{"city":"New York City","offset":"0","state_code":"NY","limit":"30","prop_type":"condo","sort":"sold_date"},
              {"city":"New York City","offset":"0","state_code":"NY","limit":"30","prop_type":"single_family","sort":"sold_date"},
              {"city":"New York City","offset":"0","state_code":"NY","limit":"30","prop_type":"multi_family","sort":"sold_date"},
              {"city":"Los Angeles","offset":"0","state_code":"CA","limit":"30","prop_type":"condo","sort":"sold_date"},
              {"city":"Los Angeles","offset":"0","state_code":"CA","limit":"30","prop_type":"single_family","sort":"sold_date"},
              {"city":"Los Angeles","offset":"0","state_code":"CA","limit":"30","prop_type":"multi_family","sort":"sold_date"},
              {"city":"Houston","offset":"0","state_code":"TX","limit":"30","prop_type":"condo","sort":"sold_date"},
              {"city":"Houston","offset":"0","state_code":"TX","limit":"30","prop_type":"single_family","sort":"sold_date"},
              {"city":"Houston","offset":"0","state_code":"TX","limit":"30","prop_type":"multi_family","sort":"sold_date"}]
    
    # querystring = {"city":"New York City","offset":"0","state_code":"NY","limit":"30","prop_type":"condo","sort":"sold_date"}

    headers = {
        'x-rapidapi-key': "b08c665c14msh68ef3f353b2ab06p1e6afdjsn13f2edd2a0f1",
        'x-rapidapi-host': "realtor.p.rapidapi.com"
        }
    
    for t in targets:
        print(t)
        querystring = t
        
        response = requests.request("GET", url, headers=headers, params=querystring)
        
        json_str = response.text
        
        save_to_db(json_str)
        print('save successful')
    
# find_list()

{'city': 'New York City', 'offset': '0', 'state_code': 'NY', 'limit': '30', 'prop_type': 'single_family', 'sort': 'sold_date'}
Records created successfully
save successful
{'city': 'New York City', 'offset': '0', 'state_code': 'NY', 'limit': '30', 'prop_type': 'multi_family', 'sort': 'sold_date'}
Records created successfully
save successful
{'city': 'Los Angeles', 'offset': '0', 'state_code': 'CA', 'limit': '30', 'prop_type': 'condo', 'sort': 'sold_date'}
Records created successfully
save successful
{'city': 'Los Angeles', 'offset': '0', 'state_code': 'CA', 'limit': '30', 'prop_type': 'single_family', 'sort': 'sold_date'}
Records created successfully
save successful
{'city': 'Los Angeles', 'offset': '0', 'state_code': 'CA', 'limit': '30', 'prop_type': 'multi_family', 'sort': 'sold_date'}
Records created successfully
save successful
{'city': 'Houston', 'offset': '0', 'state_code': 'TX', 'limit': '30', 'prop_type': 'condo', 'sort': 'sold_date'}
Records created successfully
save successfu

In [15]:
result['properties']

[{'property_id': 'O4770831613',
  'listing_id': '2921884924',
  'prop_type': 'condo',
  'list_date': '2020-09-30T03:56:37Z',
  'last_update': '2020-12-12T04:41:19Z',
  'year_built': None,
  'beds': 3,
  'baths_full': 2,
  'baths_half': None,
  'prop_status': 'not_for_sale',
  'address': {'line': '5 Roebling St Apt 5A',
   'street_number': '5',
   'street': 'Roebling',
   'street_suffix': 'St',
   'unit': 'Apt 5A',
   'city': 'Brooklyn',
   'state': 'New York',
   'state_code': 'NY',
   'postal_code': '11211',
   'lat': 40.718122,
   'county': 'Kings',
   'neighborhood_name': 'Williamsburg - Brooklyn,NY',
   'neighborhoods': [{'name': 'Williamsburg',
     'city': 'Brooklyn',
     'state_code': 'NY',
     'id': '904b59da-47d2-5cc2-927a-d86f3b99cf68',
     'level': 'neighborhood'}],
   'lon': -73.952895},
  'mls': {'abbreviation': 'RBNY', 'id': 'OLRS-771091'},
  'client_display_flags': {'presentation_status': 'recently_sold',
   'is_showcase': False,
   'lead_form_phone_required': False,


In [52]:
def read_house_detail_json(house_detail_json_str):
    try:
        detail_json = json.loads(house_detail_json_str)
        if detail_json and 'properties' in detail_json:

            p = detail_json['properties']
            if len(p) > 0:
                detail_dic = p[0]
                if 'property_id' in detail_dic and 'property_history' in detail_dic:
                    d_list = detail_dic['property_history']
                    if len(d_list) > 0:
                        insert_history_prices(detail_dic['property_id'], d_list)
    except:
      print("An exception occurred")

In [12]:

    
def find_house_detail(property_id):
    url = "https://realtor.p.rapidapi.com/properties/v2/detail"

    querystring = {"property_id":property_id}

    headers = {
        'x-rapidapi-key': "b08c665c14msh68ef3f353b2ab06p1e6afdjsn13f2edd2a0f1",
        'x-rapidapi-host': "realtor.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    return response.text

house_detail_json = find_house_detail("O4684373068")


'{"meta":{"build":"3.23.127","schema":"core.3","tracking":"type|meta|data|resource_type|property_detail|query|client_id|rdc_mobile_native,13.3.0.53|schema|core.3|tag_version|v2^^^$0|1|2|$3|4|5|$6|7|8|9|A|B]]]","returned_rows":1,"matching_rows":1,"tracking_params":{"ldpPropertyStatus":"ldp:not_for_sale","pageType":"ldp","leadDelivery":"unknown","leadEnhancements":"unknown","listingActivity":"unknown","productType":"core.agent","propertyStatus":"not_for_sale","listingId":"2918573467","rentalDataSource":"unknown","advertiserIdAgent":"1090468","advertiserIdOffice":"51370","communityId":"unknown","mprId":"4684373068","listingMls":"SINY","planId":"unknown","subId":"unknown","city":"Staten Island","neighborhood":"South Shore","state":"NY","zip":"10312","listingBaths":"2","listingBeds":"3","listingSqFt":"1148","listingEnhancements":"unknown","listingPrice":"490000","photoCount":"45","propertyType":"ldp:condo","version":"1.0"}},"properties":[{"property_id":"O4684373068","prop_status":"recently_

In [47]:
# test
#rows = query_houses()
#print(rows)

In [54]:
def load_peoperty_detail():
    
    # 1) load db
    rows = query_houses()
    
    # 2) invoke web api
    i = 0
    if len(rows) > 0:
        for row in rows:
            property_id = row[1]
            print("NO. {}, id = {}".format(i, property_id))
            
            time.sleep(3)
        
            detail_json_str = find_house_detail(property_id)
            
            # 3) insert db
            if detail_json_str:
                read_house_detail_json(detail_json_str)
            
            
            i += 1
    
    

load_peoperty_detail()

NO. 0, id = O4684373068
Records created successfully
NO. 1, id = O4972388730
Records created successfully
NO. 2, id = O3944496635
Records created successfully
NO. 3, id = O3296087243
Records created successfully
NO. 4, id = O3484025688
Records created successfully
NO. 5, id = O3645991546
Records created successfully
NO. 6, id = O3075273785
Records created successfully
NO. 7, id = O4763549378
Records created successfully
NO. 8, id = O3341300981
Records created successfully
NO. 9, id = O9064118988
Records created successfully
NO. 10, id = O3498573255
Records created successfully
NO. 11, id = O1342399516
Records created successfully
NO. 12, id = O2738107654
Records created successfully
NO. 13, id = O2017028114
Records created successfully
NO. 14, id = O1414670866
Records created successfully
NO. 15, id = O4770831613
Records created successfully
NO. 16, id = O3285464677
Records created successfully
NO. 17, id = O3770561258
Records created successfully
NO. 18, id = O9325484330
Records creat

Records created successfully
NO. 155, id = O2263450120
Records created successfully
NO. 156, id = O1589582768
Records created successfully
NO. 157, id = O1361458248
Records created successfully
NO. 158, id = O1542303659
Records created successfully
NO. 159, id = O1197045282
Records created successfully
NO. 160, id = O2423790248
Records created successfully
NO. 161, id = O2385973540
Records created successfully
NO. 162, id = O2308368178
Records created successfully
NO. 163, id = O1389945876
Records created successfully
NO. 164, id = O1000007648
Records created successfully
NO. 165, id = O2682055304
Records created successfully
NO. 166, id = O2923438706
Records created successfully
NO. 167, id = O2247767549
Records created successfully
NO. 168, id = O1157137309
Records created successfully
NO. 169, id = O1519498036
Records created successfully
NO. 170, id = O1387209868
Records created successfully
NO. 171, id = O9504223440
Records created successfully
NO. 172, id = O1964960837
Records cr

In [18]:
print(detail_json['properties'])
print(type(detail_json['properties']))

[{'property_id': 'O4684373068', 'prop_status': 'recently_sold', 'listing_id': '2918573467', 'prop_type': 'condo', 'list_date': '2020-12-10T19:19:08Z', 'hoa_fee': 225, 'last_update': '2020-12-10T12:47:36Z', 'buyer_agent_for_far': {'advertiser_id': 3705051, 'name': 'Christian Jackson', 'nrds_verified_id': None, 'mls_membership': {'member': {'id': '17211', 'agent_system_id': '17211', 'name': 'Christian Jackson', 'abbreviation': 'SINY'}}}, 'broker': {'name': 'Robert Defalco Realty, Inc.', 'phone1': {'number': '7189877900', 'type': 'broker'}}, 'year_built': 1990, 'listing_status': 'Closed', 'beds': 3, 'description': '20148R: Well maintained both inside & out, this One Family END unit home features 3 levels above ground, 3 bdrms, 2 baths, full finished basement, large back & side yard w/2 sheds, EIK w/SGD to deck/yard, attic + 2 car assigned parking spots. Heating system 1 yr young. Community features large inground pool. Description: LEVEL 1: Living/Dining combo, EIK w/SGD to deck/yard, hal

In [2]:
import requests

url = "https://realtor.p.rapidapi.com/properties/v2/list-sold"

querystring = {"city":"New York City","offset":"0","state_code":"NY","limit":"200","sort":"sold_date"}

headers = {
    'x-rapidapi-key': "b08c665c14msh68ef3f353b2ab06p1e6afdjsn13f2edd2a0f1",
    'x-rapidapi-host': "realtor.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"meta":{"build":"3.23.127","schema":"core.3","tracking_params":{"channel":"not_for_sale","siteSection":"not_for_sale","city":"New York City","county":"unknown","neighborhood":"unknown","searchCityState":"New York City, NY","state":"NY","zip":"unknown","srpPropertyStatus":"srp:not_for_sale","listingActivity":"unknown","propertyStatus":"not_for_sale","propertyType":"any","searchBathrooms":"any","searchBedrooms":"any","searchMaxPrice":"unknown","searchMinPrice":"unknown","searchRadius":"unknown","searchHouseSqft":"any","searchLotSqft":"any","searchResults":"200","sortResults":"sold_date","searchCoordinates":"unknown","version":"1.0"},"tracking":"type|meta|data|resource_type|property_list|query|client_id|rdc_mobile_native,13.3.0.53|os|13.3.1|prop_status|recently_sold|schema|core.3|limit|offset|city|New+York+City|state_code|NY|sort|sold_date|for_sale|count|total^5K|0|5K|BDB^^$0|1|2|$3|4|5|$6|7|8|9|A|B|C|D|E|P|F|Q|G|H|I|J|K|L]|A|M|N|R|O|S]]","returned_rows":200,"matching_rows":14735},"prope

{"meta":{"build":"3.23.127","schema":"core.3","tracking":"type|meta|data|resource_type|property_detail|query|client_id|rdc_mobile_native,13.3.0.53|schema|core.3|tag_version|v2^^^$0|1|2|$3|4|5|$6|7|8|9|A|B]]]","returned_rows":1,"matching_rows":1,"tracking_params":{"ldpPropertyStatus":"ldp:not_for_sale","pageType":"ldp","leadDelivery":"unknown","leadEnhancements":"unknown","listingActivity":"unknown","productType":"basic","propertyStatus":"not_for_sale","listingId":"unknown","rentalDataSource":"unknown","advertiserIdAgent":"2821356","advertiserIdOffice":"1513383","communityId":"unknown","mprId":"3599084026","listingMls":"unknown","planId":"unknown","subId":"unknown","city":"Rockaway Park","neighborhood":"Rockaway Peninsula","state":"NY","zip":"11694","listingBaths":"1","listingBeds":"2","listingSqFt":"1200","listingEnhancements":"unknown","listingPrice":"683200","photoCount":"44","propertyType":"ldp:single_family","version":"1.0"}},"properties":[{"property_id":"O3599084026","prop_status"

In [3]:
url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"

querystring = {"city":"New York City","limit":"200","offset":"0","state_code":"NY","sort":"relevance"}

headers = {
    'x-rapidapi-key': "b08c665c14msh68ef3f353b2ab06p1e6afdjsn13f2edd2a0f1",
    'x-rapidapi-host': "realtor.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"meta":{"build":"3.23.127","schema":"core.3","tracking_params":{"channel":"for_sale","siteSection":"for_sale","city":"New York City","county":"unknown","neighborhood":"unknown","searchCityState":"New York City, NY","state":"NY","zip":"unknown","srpPropertyStatus":"srp:for_sale","listingActivity":"unknown","propertyStatus":"for_sale","propertyType":"any","searchBathrooms":"any","searchBedrooms":"any","searchMaxPrice":"unknown","searchMinPrice":"unknown","searchRadius":"unknown","searchHouseSqft":"any","searchLotSqft":"any","searchResults":"200","sortResults":"relevance","searchCoordinates":"unknown","version":"1.0"},"tracking":"type|meta|data|resource_type|property_list|query|client_id|rdc_mobile_native,13.3.0.53|prop_status|for_sale|schema|core.3|limit|offset|city|New+York+City|state_code|NY|sort|relevance|count|total^5K|0|5K|SS8^^$0|1|2|$3|4|5|$6|7|8|9|A|B|C|M|D|N|E|F|G|H|I|J]|8|9|K|O|L|P]]","returned_rows":200,"matching_rows":37304},"properties":[{"property_id":"M3750418448","listin

In [5]:
# 另一个api
url = "https://realtor-com1.p.rapidapi.com/v1/buy"

querystring = {"search":"07302","list_date_max":"2020-10-01","list_price_max":"200000","year_built_min":"1980","list_price_min":"100000","sort_list_date":"desc","year_built_max":"2020","list_date_min":"2020-09-01"}

headers = {
    'x-rapidapi-key': "b08c665c14msh68ef3f353b2ab06p1e6afdjsn13f2edd2a0f1",
    'x-rapidapi-host': "realtor-com1.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

In [3]:
# zillow 的 by zip code
url = "https://zillow-free.p.rapidapi.com/properties/zipcode/94105"

querystring = {"min_price":"0","page":"1","max_price":"0"}

headers = {
    'x-rapidapi-key': "b08c665c14msh68ef3f353b2ab06p1e6afdjsn13f2edd2a0f1",
    'x-rapidapi-host': "zillow-free.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)


<!DOCTYPE html>
<html lang="en">
  <head>
  <meta charset="utf-8">
  <meta content="width=300, initial-scale=1" name="viewport">
  <meta name="google-site-verification" content="LrdTUW9psUAMbh4Ia074-BPEVmcpBxF6Gwf0MSgQXZs">
  <title>Sign in - Google Accounts</title>
  <style>
  @font-face {
  font-family: 'Open Sans';
  font-style: normal;
  font-weight: 300;
  src: url(//fonts.gstatic.com/s/opensans/v15/mem5YaGs126MiZpBA-UN_r8OUuhs.ttf) format('truetype');
}
@font-face {
  font-family: 'Open Sans';
  font-style: normal;
  font-weight: 400;
  src: url(//fonts.gstatic.com/s/opensans/v15/mem8YaGs126MiZpBA-UFVZ0e.ttf) format('truetype');
}
  </style>
  <style>
  h1, h2 {
  -webkit-animation-duration: 0.1s;
  -webkit-animation-name: fontfix;
  -webkit-animation-iteration-count: 1;
  -webkit-animation-timing-function: linear;
  -webkit-animation-delay: 0;
  }
  @-webkit-keyframes fontfix {
  from {
  opacity: 1;
  }
  to {
  opacity: 1;
  }
  }
  </style>
<style>
  html, body {
  font-fami